In [1]:
import numpy as np
import pandas as pd
import paddle
from paddle import nn
from paddle.nn import functional as F
from paddle.optimizer import Adam
from paddle.io import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sqlite3

In [2]:
paddle.device.set_device("gpu:0")

Place(gpu:0)

In [3]:
# Connect to the SQLite database
connection = sqlite3.connect("./data/data203020/binance.db")

# Define the SQL query
sql_query = "SELECT * FROM BTCUSDT"

# Read data from the SQLite database into a pandas DataFrame
df = pd.read_sql_query(sql_query, connection)

# Close the database connection
connection.close()

In [4]:
# 将字符串类型的数值列转换为浮点数
numeric_columns = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']
df[numeric_columns] = df[numeric_columns].astype(float)

In [5]:
# Calculate RSI
delta = df['close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))

# Calculate EMA
ema_short = df['close'].ewm(span=12).mean()
ema_long = df['close'].ewm(span=26).mean()

# Calculate MACD
macd = ema_short - ema_long
signal_line = macd.ewm(span=9).mean()
histogram = macd - signal_line

# Add RSI, EMA, and MACD to the DataFrame
df['rsi'] = rsi
df['ema_short'] = ema_short
df['ema_long'] = ema_long
df['macd'] = macd
df['signal_line'] = signal_line
df['histogram'] = histogram

In [6]:
df['sma_3'] = df['close'].rolling(window=3).mean()
df['sma_6'] = df['close'].rolling(window=6).mean()
df['sma_12'] = df['close'].rolling(window=12).mean()

In [7]:
# 波动率
df['volatility_std'] = df['close'].rolling(window=5).std()

In [8]:
df['pct_change'] = df['close'].pct_change()

In [9]:
df['sma_20'] = df['close'].rolling(window=20).mean()
df['std_20'] = df['close'].rolling(window=20).std()
df['bollinger_upper'] = df['sma_20'] + 2 * df['std_20']
df['bollinger_middle'] = df['sma_20']
df['bollinger_lower'] = df['sma_20'] - 2 * df['std_20']


df['diff_bollinger_upper'] = df['close'] - df['bollinger_upper']
df['diff_bollinger_lower'] = df['close'] - df['bollinger_lower']

In [10]:
df['diff_sma_3'] = df['close'] - df['sma_3']
df['diff_sma_6'] = df['close'] - df['sma_6']
df['diff_sma_12'] = df['close'] - df['sma_12']

In [11]:
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,sma_20,std_20,bollinger_upper,bollinger_middle,bollinger_lower,diff_bollinger_upper,diff_bollinger_lower,diff_sma_3,diff_sma_6,diff_sma_12
0,1504254000000,4716.47,4730.00,4716.47,4727.96,2.455540,1504254299999,11600.812852,32,1.659262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1504254300000,4728.02,4762.99,4728.02,4762.99,1.969549,1504254599999,9375.300274,18,1.683266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1504254600000,4731.12,4731.36,4731.12,4731.36,0.299831,1504254899999,1418.597288,3,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.410000,NaN,NaN
3,1504254900000,4750.00,4750.00,4746.63,4746.65,1.809314,1504255199999,8591.265774,13,0.983768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.350000,NaN,NaN
4,1504255200000,4746.65,4750.00,4746.65,4749.63,2.505225,1504255499999,11898.643549,22,1.041106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.083333,NaN,NaN


In [12]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
       'taker_buy_quote_asset_volume', 'ignore', 'rsi', 'ema_short',
       'ema_long', 'macd', 'signal_line', 'histogram', 'sma_3', 'sma_6',
       'sma_12', 'volatility_std', 'pct_change', 'sma_20', 'std_20',
       'bollinger_upper', 'bollinger_middle', 'bollinger_lower',
       'diff_bollinger_upper', 'diff_bollinger_lower', 'diff_sma_3',
       'diff_sma_6', 'diff_sma_12'],
      dtype='object')

In [13]:
data = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
       'taker_buy_quote_asset_volume', 'rsi', 'ema_short',
       'ema_long', 'macd', 'signal_line', 'histogram', 'sma_3', 'sma_6',
       'sma_12', 'volatility_std', 'pct_change', 'sma_20', 'std_20',
       'bollinger_upper', 'bollinger_middle', 'bollinger_lower',
       'diff_bollinger_upper', 'diff_bollinger_lower', 'diff_sma_3',
       'diff_sma_6', 'diff_sma_12']]

In [14]:
data.head(20)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,sma_20,std_20,bollinger_upper,bollinger_middle,bollinger_lower,diff_bollinger_upper,diff_bollinger_lower,diff_sma_3,diff_sma_6,diff_sma_12
0,1504254000000,4716.47,4730.00,4716.47,4727.96,2.455540,1504254299999,11600.812852,32,1.659262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1504254300000,4728.02,4762.99,4728.02,4762.99,1.969549,1504254599999,9375.300274,18,1.683266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1504254600000,4731.12,4731.36,4731.12,4731.36,0.299831,1504254899999,1418.597288,3,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.410000,NaN,NaN
3,1504254900000,4750.00,4750.00,4746.63,4746.65,1.809314,1504255199999,8591.265774,13,0.983768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.350000,NaN,NaN
4,1504255200000,4746.65,4750.00,4746.65,4749.63,2.505225,1504255499999,11898.643549,22,1.041106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.083333,NaN,NaN
5,1504255500000,4750.00,4750.00,4749.63,4749.63,2.949509,1504255799999,14009.757925,9,1.841874,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.993333,4.926667,NaN
6,1504255800000,4750.00,4750.00,4731.86,4731.86,6.553389,1504256099999,31086.797081,27,1.050000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.846667,-13.493333,NaN
7,1504256100000,4740.93,4749.99,4731.00,4749.99,4.328167,1504256399999,20503.587562,18,0.762278,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.163333,6.803333,NaN
8,1504256400000,4731.01,4749.99,4731.01,4747.97,2.658644,1504256699999,12618.893283,11,0.613064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.696667,2.015000,NaN
9,1504256700000,4745.94,4749.99,4731.00,4749.99,2.411020,1504256999999,11438.103835,21,0.734977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.673333,3.478333,NaN


In [15]:
# 丢弃前20行含有nan的数据
data = data.iloc[20:]

data.reset_index(drop=True, inplace=True)

In [16]:
data.isna().sum()

open_time                       0
open                            0
high                            0
low                             0
close                           0
volume                          0
close_time                      0
quote_asset_volume              0
number_of_trades                0
taker_buy_base_asset_volume     0
taker_buy_quote_asset_volume    0
rsi                             5
ema_short                       0
ema_long                        0
macd                            0
signal_line                     0
histogram                       0
sma_3                           0
sma_6                           0
sma_12                          0
volatility_std                  0
pct_change                      0
sma_20                          0
std_20                          0
bollinger_upper                 0
bollinger_middle                0
bollinger_lower                 0
diff_bollinger_upper            0
diff_bollinger_lower            0
diff_sma_3    

In [17]:
data = data.fillna(0)
data.isna().sum()

open_time                       0
open                            0
high                            0
low                             0
close                           0
volume                          0
close_time                      0
quote_asset_volume              0
number_of_trades                0
taker_buy_base_asset_volume     0
taker_buy_quote_asset_volume    0
rsi                             0
ema_short                       0
ema_long                        0
macd                            0
signal_line                     0
histogram                       0
sma_3                           0
sma_6                           0
sma_12                          0
volatility_std                  0
pct_change                      0
sma_20                          0
std_20                          0
bollinger_upper                 0
bollinger_middle                0
bollinger_lower                 0
diff_bollinger_upper            0
diff_bollinger_lower            0
diff_sma_3    

In [18]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# 创建滑动窗口
def create_sliding_window(data, window_size, target_col_index):
    x, y = [], []
    for i in range(len(data) - window_size):
        x.append(data[i:i+window_size, :])
        y.append(data[i+window_size, target_col_index])
    return np.array(x), np.array(y)

# 设定窗口大小（例如：30个时间步长）
window_size = 30
target_col_index = 4  # 假设目标列（如 'close'）在第4列

X, y = create_sliding_window(data_scaled, window_size, target_col_index)

# 数据划分
train_ratio = 0.8
train_size = int(len(X) * train_ratio)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 将数据转换为PaddlePaddle所需的格式
X_train, y_train = paddle.to_tensor(X_train).astype('float32'), paddle.to_tensor(y_train).astype('float32')
X_test, y_test = paddle.to_tensor(X_test).astype('float32'), paddle.to_tensor(y_test).astype('float32')

W0412 13:01:36.692705   416 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0412 13:01:36.696614   416 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


In [19]:
# 构建自定义模型
class CustomModel(nn.Layer):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomModel, self).__init__()

        self.bidirectional_lstm1 = nn.LSTM(input_size, hidden_size, direction="bidirectional")
        self.bidirectional_lstm2 = nn.LSTM(hidden_size * 2, hidden_size, direction="bidirectional")
        self.pooling = nn.AdaptiveAvgPool1D(1)
        self.unidirectional_lstm = nn.LSTM(hidden_size * 2, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.bidirectional_lstm1(x)
        x, _ = self.bidirectional_lstm2(x)
        x = self.pooling(x.transpose([0, 2, 1])).transpose([0, 2, 1])
        x, _ = self.unidirectional_lstm(x)
        x = self.fc(x[:, -1, :])
        return x

# 模型参数设置
input_size = X_train.shape[2]
hidden_size = 64
output_size = 1

# 初始化模型、损失函数、优化器
model = CustomModel(input_size, hidden_size, output_size)
loss_fn = nn.MSELoss()
optimizer = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())

In [20]:
# 训练参数设置
epochs = 1000
batch_size = 128

import os

# 定义模型保存路径
model_save_dir = 'saved_models'
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

# 训练模型
for epoch in range(epochs):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)

        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
    print(f'Epoch {epoch + 1}, Loss: {loss.numpy()[0]}')
    # 每100轮保存一次模型
    if (epoch + 1) % 100 == 0:
        model_path = os.path.join(model_save_dir, f'epoch_{epoch + 1}_model.pdparams')
        paddle.save(model.state_dict(), model_path)
        print(f'Model saved at epoch {epoch + 1}: {model_path}')

Epoch 1, Loss: 3.827757609542459e-05


Epoch 2, Loss: 0.0007242051651701331


Epoch 3, Loss: 0.002838626503944397


Epoch 4, Loss: 0.0022480678744614124


Epoch 5, Loss: 0.0030300337821245193


...

Epoch 996, Loss: 6.725580078636995e-06


Epoch 997, Loss: 6.23468213234446e-06


Epoch 998, Loss: 3.6709313917526742e-06


Epoch 999, Loss: 5.3037042562209535e-06


Epoch 1000, Loss: 1.3100861906423233e-06
Model saved at epoch 1000: saved_models/epoch_1000_model.pdparams


In [21]:
# paddle.device.set_device("cpu")

In [22]:
# # 加载模型
model = CustomModel(input_size, hidden_size, output_size)
model_state_dict = paddle.load('btc_custom_model.pdparams')
model.load_dict(model_state_dict)
model.eval()

# 使用测试数据进行预测
y_pred = model(X_test)

# 将预测结果转换为Numpy数组
y_pred_np = y_pred.numpy()

# 反归一化预测结果
y_pred_unscaled = scaler.inverse_transform(np.concatenate((X_test[:, -1, :].numpy(), y_pred_np), axis=1))[:, -1]

# 计算评估指标（如：RMSE）
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test.numpy(), y_pred_np))
print(f'RMSE: {rmse}')

# 可视化预测结果
import matplotlib.pyplot as plt

plt.plot(y_pred_unscaled, label='Predicted')
plt.plot(scaler.inverse_transform(np.concatenate((X_test[:, -1, :].numpy(), y_test.numpy()), axis=1))[:, -1], label='Actual')
plt.xlabel('Time Step')
plt.ylabel('Close Price')
plt.legend()
plt.show()

ValueError: The ``path`` (btc_custom_model.pdparams) to load model not exists.